In [1]:
import os
import numpy as np
path = os.getcwd()
from netCDF4 import Dataset

import matplotlib.pyplot as plt
import xgboost as xgb

# In[]
lr = 1e-3
batch_size = 4
num_epoch = 200
ckpt_dir = './checkpoint'

In [3]:
# In[] SMOKE-CMAQ
# CMAQ 연평균 PM농도
pm2_5 = np.array([])

for i in range(1,120):
    a = 'ACONC.' + str(i)
    nc = Dataset(a, 'r')
    pm2_5 = np.append(pm2_5, np.array([[[nc.variables['PM2_5']]]]))
    
pm2_5 = np.reshape(pm2_5, [119,82,67])
pm2_5 = pm2_5[:,8:-10,2:-1]
pm2_5 = np.reshape(pm2_5, [119,64,64]) # convolution 연산을 위해

pm = pm2_5


y1 = np.array([])
y2 = np.array([])
y3 = np.array([])
y4 = np.array([])
y5 = np.array([])
y6 = np.array([])

for i in range(1,120):
    a = 'EMIS_AVG.' + str(i)
    nc = Dataset(a, 'r')
    y1 = np.append(y1, np.array([[[nc.variables['SO2']]]]))
    y2 = np.append(y2, np.array([[[nc.variables['PM2_5']]]]))
    y3 = np.append(y3, np.array([[[nc.variables['NOx']]]]))
    y4 = np.append(y4, np.array([[[nc.variables['VOCs']]]]))
    y5 = np.append(y5, np.array([[[nc.variables['NH3']]]]))
    y6 = np.append(y6, np.array([[[nc.variables['CO']]]]))
    
y1 = np.reshape(y1, [119,82,67])
y1 = y1[:,8:-10,2:-1]
y1 = np.reshape(y1, [119,64,64])
y1_max = np.max(y1)
y1_min = np.min(y1)

y2 = np.reshape(y2, [119,82,67])
y2 = y2[:,8:-10,2:-1]
y2 = np.reshape(y2, [119,64,64])
y2_max = np.max(y2)
y2_min = np.min(y2)

y3 = np.reshape(y3, [119,82,67])
y3 = y3[:,8:-10,2:-1]
y3 = np.reshape(y3, [119,64,64])
y3_max = np.max(y3)
y3_min = np.min(y3)

y4 = np.reshape(y4, [119,82,67])
y4 = y4[:,8:-10,2:-1]
y4 = np.reshape(y4, [119,64,64])
y4_max = np.max(y4)
y4_min = np.min(y4)

y5 = np.reshape(y5, [119,82,67])
y5 = y5[:,8:-10,2:-1]
y5 = np.reshape(y5, [119,64,64])
y5_max = np.max(y5)
y5_min = np.min(y5)
    
y6 = np.reshape(y6, [119,82,67])
y6 = y6[:,8:-10,2:-1]
y6 = np.reshape(y6, [119,64,64])


def pm_data():
    a = np.concatenate((y3,y1,y4,y5), axis=0) # PM2.5, SO2, NH3
    a = np.reshape(a,[4,119,64,64]) # data generation for CNN ??
    a = np.transpose(a, (1,2,3,0)) # CNN 학습을 위한 순서변경
    print(np.shape(a))
    
    y = pm
    
    y = np.reshape(y, [119,64,64,1]) # CNN output data
    x_train = a[0:70]
    y_train = y[0:70]
    x_test = a[70:]
    pm2_5 = y[70:]
    return x_train, y_train, x_test, pm2_5, a, y


pm_data1 = pm_data()
x_train, y_train, x_test, pm2_5, a, y = pm_data1[0], pm_data1[1], pm_data1[2], pm_data1[3], pm_data1[4], pm_data1[5]#, pm_data1[6], pm_data1[7] 


(119, 64, 64, 4)


In [36]:
x_train_spread = x_train.reshape(x_train.shape[0],-1)
x_test_spread = x_test.reshape(x_test.shape[0],-1)

y_train_spread = y_train.reshape(y_train.shape[0],-1)
y_test_spread = pm2_5.reshape(pm2_5.shape[0],-1)
# np.sum(x_train - test.reshape(70,64,64,4))

print(x_train_spread.shape, x_test_spread.shape, y_train_spread.shape, y_test_spread.shape, )

(70, 16384) (49, 16384) (70, 4096) (49, 4096)


In [ ]:
%%time
params = {
        # 'booster':'gbtree', # [gbtree, gblinear, dart]
        "tree_method": "hist", 
        'objective':'reg:squaredlogerror',
        # 'objective':'reg:squaredlogerror',binary:logistic
        "verbosity": 0,
                }
train_list = []
test_list = []
pred_list = []

for i in range(len(y_train_spread[0,:])):
    early_stop = xgb.callback.EarlyStopping(rounds=20,
                                            data_name='val'
                                            )

    dtrain_train = xgb.DMatrix(data=x_train_spread, label=y_train_spread[:,i])
    dval_train = xgb.DMatrix(data=x_test_spread, label=y_test_spread[:,i])

    train_list.append(dtrain_train)
    test_list.append(dval_train)

    model = xgb.train(params,
                    dtrain_train,
                    evals=[(dtrain_train,'train'),(dval_train,'val')],
                    callbacks = [early_stop],
                    num_boost_round=100,)

    y_pred = xgb.Booster.predict(model,dval_train)
    pred_list.append(y_pred)
    